# Phase 2, 3 and 4: Search, Augment and Generate the Answer
In this notebook there are several parts:
- Import libraries, load configuration variables and create clients
- Hybrid search with Semantic ranker
- Filter the chunks leaving the most relevant compared with the user's question
- Generate the answer for the query using the most relevante chunks as the context

### Import libraries, load configuration variables and create clients

In [ ]:
#%pip install python-dotenv
#%pip install openai
#%pip install tiktoken
#%pip install azure-search-documents

In [ ]:
# Import libraries
import os
import sys
import json
import time
import pandas as pd
from dotenv import load_dotenv, find_dotenv
from openai import AzureOpenAI

sys.path.append(os.path.abspath('..'))
from common_utils import *

# Load Azure OpenAI and AI Search variables and create clients
openai_config, ai_search_config = load_config()

# Prepare AI Search client
# We will use the 'docs' index for this example
ai_search_client = SearchClient(endpoint=ai_search_config["ai_search_endpoint"],
                                index_name=ai_search_config["ai_search_index_name_docs"],
                                credential=AzureKeyCredential(ai_search_config["ai_search_apikey"]))

## Process Step by Step
1. Search in AI Search
2. Filter relevant chunks
3. Generate answer

### 1. Search in AI Search with hybrid (keyword and vector searches) with semantic ranker

In [ ]:
# Generate the query for the question
question = "What is included in my Northwind Health Plus plan?"

# Hybrid search
results, num_results = semantic_hybrid_search(ai_search_client=ai_search_client,
                                              openai_client=openai_config["openai_client"],
                                              aoai_embedding_model=openai_config["aoai_embedding_model"],
                                              query=question,
                                              max_docs=10)
print(f"num results: {num_results}")
print(f"num len(results): {len(results)}")
show_results(results, question)


#### 2. Filter the chunks compared with the user's question

In [ ]:
# Valid chunks for the user question
valid_chunks, num_chunks = get_filtered_chunks(openai_config["openai_client"],
                                               openai_config["aoai_rerank_model"],
                                               results,
                                               question)
print(f"num valid chunks: {num_chunks}")


#### 3. Generate the answer using the relevant chunks as context

In [ ]:
# Generate answer:
answer = generate_answer(openai_config["openai_client"],
                         openai_config["aoai_deployment_name"],
                                       valid_chunks,
                                       question)
print(f"\n>> Answer:\n{answer}")


## End-to-end process

In [ ]:
## End-to-end process:

question = "What is included in my Northwind Health Plus plan?"
print(f'Question: {question}')

# Hybrid search with Semantic ranker
results, num_results = semantic_hybrid_search(ai_search_client=ai_search_client,
                                              openai_client=openai_config["openai_client"],
                                              aoai_embedding_model=openai_config["aoai_embedding_model"],
                                              query=question,
                                              max_docs=50)
print(f"num results: {num_results}")
show_results(results, question)

# Filter valid chunks for the user question
valid_chunks, num_chunks = get_filtered_chunks(openai_config["openai_client"],
                                               openai_config["aoai_rerank_model"],
                                               results, question)

# Generate answer:
answer = generate_answer(openai_config["openai_client"],
                         openai_config["aoai_deployment_name"],
                         valid_chunks, question)
print(f"\n>> Answer: {answer}")

## End to End Process Using conversation history

In [ ]:
## End-to-end process using conversation history:

import pandas as pd

# Read test data from Excel file
input_file = "../5_evaluation/ground_truth.xlsx"
df = pd.read_excel(input_file,)
data_dict = df.to_dict(orient='records')

question = ''
history=[]
for i, line in enumerate(data_dict):

    question = line['QUESTION']

    print(f'[{i+1}] Question: {question}')
    query = generate_search_query(openai_config["openai_client"],
                           openai_config["aoai_deployment_name"],
                           question,
                           history)
    print(f'Rewritten Question: {query}')

    # Hybrid search with Semantic ranker
    results, num_results = semantic_hybrid_search(ai_search_client=ai_search_client,
                                                  openai_client=openai_config["openai_client"],
                                                  aoai_embedding_model=openai_config["aoai_embedding_model"],
                                                  query=query,
                                                  max_docs=50)
    print(f"num results: {num_results}")
    #show_results(results, query)

    # Filter valid chunks for the user question
    valid_chunks, num_chunks = get_filtered_chunks(openai_config["openai_client"],
                                                openai_config["aoai_rerank_model"],
                                                results, question)
    # Generate answer with best chunks as context and the conversation history:
    answer = generate_answer_with_history(openai_config["openai_client"],
                                          openai_config["aoai_deployment_name"],
                                          valid_chunks,
                                          question,
                                          history)
    print(f"\n>> Answer: {answer}\n")

    # check if the number of question and answer pair has reached the limit of 3 and remove the oldest one
    if len(history) >= 3:
        history.pop(0)
    history.append({"question": question, "answer": answer})
    print(f"\nhistory: {json.dumps(history, indent=2)}\n")
    print("--------------------------------------------------")